In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv, find_dotenv
from pyairtable import Table

config = load_dotenv(find_dotenv())

In [ ]:
pd.options.display.max_rows = None
plt.rc("figure", figsize=[12, 4])

### Import des données brutes

In [ ]:
# Fetch data from Airtable API
table = Table(
    api_key=os.getenv("API_KEY"),
    base_id=os.getenv("BASE_ID"),
    table_name=os.getenv("TABLE_NAME")
)

# Loading into a dataframe
data = [col['fields'] for col in table.all()]
raw_df = pd.DataFrame(data)

### Aperçu

In [ ]:
raw_df.sample(3)

In [ ]:
desc_df = raw_df.describe().drop(['top'], axis=0).transpose()
desc_df.columns = ['Décompte', 'Valeurs uniques', 'Décompte valeur de la plus fréquente']
desc_df

### Nettoyage

In [ ]:
# Date column to datetime
raw_df[['Créé le', 'Modifié le']] = raw_df[['Créé le', 'Modifié le']].apply(pd.to_datetime, errors='coerce')
raw_df = raw_df.replace(["", np.nan], None)

### Taux de remplissage des champs de structures

In [ ]:
def compute_field_occupancy_rates(df):
    return ((1 - df.isnull().sum() / df.shape[0]) * 100).sort_values(ascending=False)

compute_field_occupancy_rates(raw_df).to_frame()

### Distribution du premier référencement

In [ ]:
sns.set(style="darkgrid")
sns.histplot(data=raw_df, x='Créé le', bins=20)

### Distribution de la dernière mise à jour

In [ ]:
sns.histplot(data=raw_df, x='Modifié le', bins=10)

### Typologie

In [ ]:
sns.countplot(
    data=raw_df.fillna('Type inconnu'),
    y='Type',
    order = raw_df['Type'].fillna('Type inconnu').value_counts().index
)

### Répartition géographique

In [ ]:
sns.countplot(
    data=raw_df.fillna('Inconnu'),
    y='Région Nom',
    order = raw_df['Région Nom'].fillna('Inconnu').value_counts().index
)

### Répartition des partenaires

In [ ]:
sns.countplot(
    data=raw_df.fillna('Inconnu'),
    y='Partenaire Nom',
    order = raw_df['Partenaire Nom'].fillna('Inconnu').value_counts().index
)

### Répartition des services

In [ ]:
def get_unique(df, col):
    unique_values = []
    for el in df[col]:
        if not el is None:
            for sub_el in el:
                if not sub_el in unique_values:
                    unique_values.append(sub_el)
    return unique_values

In [ ]:
print("Services : ", get_unique(raw_df, 'Services'))
print("Véhicules traités : ", get_unique(raw_df, 'Types de véhicule'))
print("% sans services indiqués : ",
      raw_df['Services'].isna().sum()/raw_df['Services'].count()*100)
print("% sans véhicules indiqués : ",
      raw_df['Types de véhicule'].isna().sum()/raw_df['Types de véhicule'].count())